In [7]:
import os
import pandas as pd
import numpy as np

In [8]:
raw = pd.read_excel(
  '../../data/reference/qualifications_2021census.xlsx',
  sheet_name='quals_con'
).rename(
    columns={'ONSConstID': 'constituency_code',
            'ConstituencyName': 'constituency_name',
            'variables': 'variable',
            'Con_num': 'value',
            'Con_pc': 'percent'}).set_index([
            'constituency_code', 
            'constituency_name']).drop(
    columns={'NatComparator', 
            'RegNationID',
            'RegNationName',
            'RN_pc', 
            'Nat_pc', 
            'ranking_total',
            'rank'})

raw.columns = raw.columns.str.strip()
raw

variable  \
constituency_code constituency_name                                            
E14000530         Aldershot                                No qualifications   
                  Aldershot                               1 to 4 GCSE passes   
                  Aldershot                            5 or more GCSE passes   
                  Aldershot                                   Apprenticeship   
                  Aldershot                               2 or more A levels   
...                                                                      ...   
W07000080         Cardiff South and Penarth            5 or more GCSE passes   
                  Cardiff South and Penarth                   Apprenticeship   
                  Cardiff South and Penarth               2 or more A levels   
                  Cardiff South and Penarth  Higher education qualifications   
                  Cardiff South and Penarth             Other qualifications   

                                                                      groups  \
constituency_code constituency_name                                            
E14000530         Aldershot                                No qualifications   
                  Aldershot                                  1 or more GCSEs   
                  Aldershot                                  1 or more GCSEs   
                  Aldershot                                   Apprenticeship   
                  Aldershot                               2 or more A levels   
...                                                                      ...   
W07000080         Cardiff South and Penarth                  1 or more GCSEs   
                  Cardiff South and Penarth                   Apprenticeship   
                  Cardiff South and Penarth               2 or more A levels   
                  Cardiff South and Penarth  Higher education qualifications   
                  Cardiff South and Penarth             Other qualifications   

                                             value   percent  
constituency_code constituency_name                           
E14000530         Aldershot                  16148  0.178468  
                  Aldershot                  10772  0.119053  
                  Aldershot                  12998  0.143654  
                  Aldershot                   5148  0.056896  
                  Aldershot                  15712  0.173650  
...                                            ...       ...  
W07000080         Cardiff South and Penarth  11193  0.122620  
                  Cardiff South and Penarth   4142  0.045376  
                  Cardiff South and Penarth  13653  0.149569  
                  Cardiff South and Penarth  34262  0.375342  
                  Cardiff South and Penarth   2572  0.028176  

[4011 rows x 4 columns]

In [9]:
os.makedirs('../../data/processed/yff/', exist_ok=True)
raw.to_csv('../../data/processed/yff/census_qualifications.csv')

### Census 2021 - Qualifications by ethnic group, local authority breakdown

In [10]:
census_ethnic_group = pd.read_csv(
    '../../data/reference/census-qualifications-ethnic-group.csv', index_col=0
)

variables = { 'total', 'asian', 'black_british', 'mixed_multiple', 'white', 'other' }

In [11]:

census_ethnic_group.columns = census_ethnic_group.columns.str.lower()

census_ethnic_group = census_ethnic_group.iloc[2:]

census_ethnic_group = census_ethnic_group.rename_axis('local_authority', axis=0)


data = pd.MultiIndex.from_product([['total','asian', 'black_british', 'mixed_multiple', 'white', 'other'],
                                     ['total','no_qualification','level_1', 'level_2', 'apprenticeship','level_3', 'level_4', 'other']],
                                    names=['ethnic_group','qualification'])

census_ethnic_group.columns = data
census_ethnic_group.to_csv('../../data/processed/census/qualifications_local_authority.csv')
# census_ethnic_group.loc[:,('Q1', 'Sell'):('Q2', 'Buy')]

In [12]:
def calculate_rates(data, variable):
    df = data.loc[:,(variable)]
    df = df.replace('',np.nan).astype(float)
    df = df.div(df.total, axis=0).mul(100).round(1).drop(columns='total')
    return df

# TO DO: Add rates for all ethnic breakdowns
total = calculate_rates(census_ethnic_group, 'total').to_csv('../../data/processed/census/census_qualifications_la_total.csv')